In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
# from shapely.ops import split, snap, nearest_points
import networkx as nx

# import netana as na
from netana.graph_manager.network import split_intersecting_lines, create_gdf_network
from netana.graph_manager.graph_creation import graph_from_gdf


In [2]:
network_crs = "3857"
df_streets = gpd.read_file("./notebook/street_resistencia_corrientes_v3.shp")
df_streets = df_streets.dissolve()

# Create network GeoDataFrame
- id, length and weight field
- optionally a label field
- defined crs 

In [3]:
# topology
df_streets = split_intersecting_lines(df_streets)
# create network
gdf_network = create_gdf_network(df_streets, network_crs, v_speed=4.5, v_label='calle')
print(gdf_network.crs)
print(len(gdf_network))
gdf_network.head()

EPSG:3857
47


,id,length,weight,label,geometry
0,e_1,458.76,6.116800,calle,"LINESTRING (-6546884.300 -3184165.694, -654703..."
1,e_2,459.53,6.127067,calle,"LINESTRING (-6546859.721 -3184019.784, -654700..."
2,e_3,460.80,6.144000,calle,"LINESTRING (-6547295.180 -3183823.596, -654714..."
3,e_4,471.56,6.287467,calle,"LINESTRING (-6546820.169 -3183766.855, -654695..."
4,e_5,461.49,6.153200,calle,"LINESTRING (-6547241.580 -3183488.333, -654711..."


# Create LINES and POINTS DATASETS; and GRAPH

In [4]:
G, gdf_nodes, gdf_edges  = graph_from_gdf(gdf_network)

gdf_edges.to_file("./output/pruebas_2025/edges.shp")
gdf_nodes.to_file("./output/pruebas_2025/nodes.shp")
G

# SOURCES

In [6]:
# create a clean sources gdf
def create_sources(gdf_sources, v_crs_proj, v_label=None):
    gdf_sources_2 = gdf_sources.to_crs(v_crs_proj)
    
    if 'id' not in gdf_sources_2.columns:
        gdf_sources_2['id'] = list(gdf_sources_2.index)
        gdf_sources_2['id'] = gdf_sources_2.apply(lambda row: "loc_"+str(row.id), axis=1) 

    gdf_sources_2 = gdf_sources_2[['id', 'geometry']]

    if v_label:
        gdf_sources_2['label'] = v_label
        gdf_sources_2 = gdf_sources_2[['id', 'label', 'geometry']]
        
    return gdf_sources_2

In [7]:
sources_file_path = "./notebook/source_corrientes.shp"
gdf_locations = gpd.read_file(sources_file_path)
gdf_locations = gdf_locations.explode(index_parts=True).reset_index(drop=True)

gdf_locations = create_sources(gdf_locations, network_crs, v_label='location')
#gdf_locations = gdf_locations.to_crs(network_crs)
print(gdf_locations.crs)
gdf_locations.head()

EPSG:3857


,id,label,geometry
0,1,location,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,2,location,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


In [8]:
""" nodes_in = gdf_nodes.sjoin(gpd.GeoDataFrame(geometry=gdf_locations.buffer(30)))
nodes_in = nodes_in.drop_duplicates(subset='id').reset_index(drop=True)
node_list = list(nodes_in.id)
node_list """

" nodes_in = gdf_nodes.sjoin(gpd.GeoDataFrame(geometry=gdf_locations.buffer(30)))\nnodes_in = nodes_in.drop_duplicates(subset='id').reset_index(drop=True)\nnode_list = list(nodes_in.id)\nnode_list "

In [9]:
def find_closest_node_to_source(gdf_sources, gdf_nodes):
    gdf_amenities = gpd.sjoin_nearest(gdf_sources, gdf_nodes, how='inner')
    #gdf_amenities.rename(columns={'id_left':'id', 'id_right':'node_id'}, inplace=True)
    gdf_amenities = gdf_amenities[['id', 'node_id', 'geometry']]
    return gdf_amenities

gdf_locations = find_closest_node_to_source(gdf_locations, gdf_nodes)
gdf_locations

,id,node_id,geometry
0,1,n_20,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,2,n_27,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


## TO DO - conect locations to graph

In [10]:
from shapely.geometry import MultiPoint, Point, LineString
from shapely.ops import nearest_points
search_radio = 10
# gdf_locations


edges_lists = []
if gdf_locations.geom_type[0] == 'Polygon':
    # creates the location's nodes
    gdf_loc_centroid = gdf_locations.copy()
    gdf_loc_centroid.geometry = gdf_locations.centroid
    gdf_loc_centroid['pt_cat'] = 'cen'
    gdf_loc_centroid['node_id'] = gdf_loc_centroid['id'].astype(str) +"_cen"

    gdf_loc_exterior = gdf_locations.copy()
    gdf_loc_exterior.geometry = gdf_loc_exterior.geometry.apply(lambda x: MultiPoint(list(x.exterior.coords)))
    gdf_loc_exterior = gdf_loc_exterior.explode(index_parts=False).reset_index(drop=True).reset_index()
    gdf_loc_exterior.drop_duplicates(subset='geometry', inplace=True)
    gdf_loc_exterior.reset_index(drop=True, inplace=True)
    gdf_loc_exterior['pt_cat'] = 'vertex'
    gdf_loc_exterior['node_id'] = gdf_loc_exterior['id'].astype(str) +"_v"+ gdf_loc_exterior['index'].astype(str)

    gdf_loc_points = pd.concat([gdf_loc_centroid, gdf_loc_exterior])

    # create the location's edges
    gdf_loc_edges = gdf_loc_points.loc[gdf_loc_points.pt_cat=='vertex', ['node_id', 'pt_cat']].copy()
    gdf_loc_edges['id'] = "le_"+ (gdf_loc_edges.index+1).astype(str)
    gdf_loc_edges['nd_st'] = gdf_loc_points.loc[gdf_loc_points.pt_cat=='vertex'].apply(lambda row: list(gdf_loc_points.loc[gdf_loc_points.id ==  row.id, 'node_id'])[0], axis=1)
    gdf_loc_edges['nd_en'] = gdf_loc_points.loc[gdf_loc_points.pt_cat=='vertex', 'node_id']
    gdf_loc_edges['weight'] = 0
    gdf_loc_edges['geometry'] = gdf_loc_edges.apply(lambda row: LineString([list(gdf_loc_points.loc[gdf_loc_points.node_id == row.nd_st, 'geometry'])[0], 
                                                list(gdf_loc_points.loc[gdf_loc_points.node_id == row.nd_en, 'geometry'])[0]]), 
                                                axis=1)
    gdf_loc_edges = gpd.GeoDataFrame(gdf_loc_edges, geometry='geometry')
    gdf_loc_edges.crs = gdf_locations.crs
    gdf_loc_edges = gdf_loc_edges[['id', 'nd_st', 'nd_en', 'weight', 'geometry']]
    edges_lists.append(gdf_loc_edges)

    gdf_loc_points_cen = gdf_loc_points.loc[gdf_loc_points.pt_cat!='vertex']
    gdf_loc_points = gdf_loc_points.loc[gdf_loc_points.pt_cat=='vertex']

# If only points, start from here
gdf_loc_points_2 = gpd.sjoin_nearest(gdf_loc_points, gdf_edges[['id', 'geometry']].rename(columns={'id':'id_line'}))
gdf_loc_points_2 = gdf_loc_points_2.merge(gdf_edges.rename(columns={'id':'id_line', 'geometry':'geometry_line'}), on='id_line', how='left')
gdf_loc_points_2['near_pt'] = gdf_loc_points_2.apply(lambda row: nearest_points(row.geometry, row.geometry_line)[1], axis=1)
gdf_loc_points_2['near_len'] = gdf_loc_points_2.apply(lambda row: row.near_pt.distance(row.geometry), axis=1)
gdf_loc_points_3 = gdf_loc_points_2.loc[gdf_loc_points_2.near_len <= search_radio]

# create the location's edges
gdf_loc_edges_2 = gdf_loc_points_3.copy()
gdf_loc_edges_2['id'] = "ce_"+ (gdf_loc_edges_2.index+1).astype(str)
gdf_loc_edges_2['nd_st'] = gdf_loc_edges_2['node_id']
gdf_loc_edges_2['nd_en'] = gdf_loc_edges_2['node_id']+"_cp"
gdf_loc_edges_2['weight'] = 0
gdf_loc_edges_2['geometry'] = gdf_loc_edges_2.apply(lambda row: LineString([row.geometry, row.near_pt]), axis=1)                                   
gdf_loc_edges_2 = gpd.GeoDataFrame(gdf_loc_edges_2, geometry='geometry')
gdf_loc_edges_2.crs = gdf_loc_points_3.crs
gdf_loc_edges_2 = gdf_loc_edges_2[['id', 'nd_st', 'nd_en', 'weight', 'geometry']]
edges_lists.append(gdf_loc_edges_2)

gdf_loc_points_3['len_m_A'] = gdf_loc_points_3.apply(lambda row: row.near_pt.distance(Point(row.geometry_line.coords[0])), axis=1)
gdf_loc_points_3['len_m_B'] = gdf_loc_points_3.apply(lambda row: row.near_pt.distance(Point(row.geometry_line.coords[-1])), axis=1)
gdf_loc_points_3['len_total'] = gdf_loc_points_3['len_m_A'] + gdf_loc_points_3['len_m_B']
gdf_loc_points_3['weight_A'] = gdf_loc_points_3['weight'] * gdf_loc_points_3['len_m_A'] / gdf_loc_points_3['len_total']
gdf_loc_points_3['weight_B'] = gdf_loc_points_3['weight'] * gdf_loc_points_3['len_m_B'] / gdf_loc_points_3['len_total']

# create the location's edges
gdf_loc_edges_3_A = gdf_loc_points_3.copy()
gdf_loc_edges_3_A['id'] = "cwe_"+ (gdf_loc_edges_3_A.index+1).astype(str)
gdf_loc_edges_3_A['nd_en'] = gdf_loc_edges_3_A['nd_st']
gdf_loc_edges_3_A['nd_st'] = gdf_loc_edges_3_A['node_id']+"_cp"
gdf_loc_edges_3_A['weight'] = gdf_loc_edges_3_A['weight_A']
gdf_loc_edges_3_A['geometry'] = gdf_loc_edges_3_A.apply(lambda row: LineString([row.near_pt, row.geometry_line.coords[0]]), axis=1)                              
gdf_loc_edges_3_A = gpd.GeoDataFrame(gdf_loc_edges_3_A, geometry='geometry')
gdf_loc_edges_3_A.crs = gdf_loc_edges_3_A.crs
gdf_loc_edges_3_A = gdf_loc_edges_3_A[['id', 'nd_st', 'nd_en', 'weight', 'geometry']]
edges_lists.append(gdf_loc_edges_3_A)

# create the location's edges
gdf_loc_edges_3_B = gdf_loc_points_3.copy()
gdf_loc_edges_3_B['id'] = "cwe_"+ (gdf_loc_edges_3_B.index+1).astype(str)
gdf_loc_edges_3_B['nd_st'] = gdf_loc_edges_3_B['node_id']+"_cp"
gdf_loc_edges_3_B['nd_en'] = gdf_loc_edges_3_B['nd_en']
gdf_loc_edges_3_B['weight'] = gdf_loc_edges_3_B['weight_B']
gdf_loc_edges_3_B['geometry'] = gdf_loc_edges_3_B.apply(lambda row: LineString([row.near_pt, row.geometry_line.coords[1]]), axis=1)                                
gdf_loc_edges_3_B = gpd.GeoDataFrame(gdf_loc_edges_3_B, geometry='geometry')
gdf_loc_edges_3_B.crs = gdf_loc_edges_3_B.crs
gdf_loc_edges_3_B = gdf_loc_edges_3_B[['id', 'nd_st', 'nd_en', 'weight', 'geometry']]
edges_lists.append(gdf_loc_edges_3_B)

gdf_loc_edges_merged = pd.concat(edges_lists)
# gdf_loc_edges_merged.to_file("./output/pruebas_2025/locations_edges.shp")
gdf_loc_edges_merged

c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingW

,id,nd_st,nd_en,weight,geometry
0,le_1,1_cen,1_v0,0.000000,"LINESTRING (-6546857.455 -3184136.638, -654687..."
1,le_2,1_cen,1_v1,0.000000,"LINESTRING (-6546857.455 -3184136.638, -654686..."
2,le_3,1_cen,1_v2,0.000000,"LINESTRING (-6546857.455 -3184136.638, -654683..."
3,le_4,1_cen,1_v3,0.000000,"LINESTRING (-6546857.455 -3184136.638, -654685..."
4,le_5,2_cen,2_v6,0.000000,"LINESTRING (-6546612.287 -3183678.335, -654662..."
5,le_6,2_cen,2_v7,0.000000,"LINESTRING (-6546612.287 -3183678.335, -654660..."
6,le_7,2_cen,2_v8,0.000000,"LINESTRING (-6546612.287 -3183678.335, -654660..."
7,le_8,2_cen,2_v9,0.000000,"LINESTRING (-6546612.287 -3183678.335, -654661..."
0,ce_1,1_v0,1_v0_cp,0.000000,"LINESTRING (-6546874.064 -3184160.022, -654687..."
1,ce_2,1_v3,1_v3_cp,0.000000,"LINESTRING (-6546851.446 -3184161.296, -654685..."


In [11]:
edgelist_loc = list(gdf_loc_edges_merged.apply(lambda row: ( row.nd_st, row.nd_en, {'id':row.id, 'weight': row.weight}), axis=1))
print(len(edgelist_loc))
edgelist_loc

26


[('1_cen', '1_v0', {'id': 'le_1', 'weight': 0.0}),
 ('1_cen', '1_v1', {'id': 'le_2', 'weight': 0.0}),
 ('1_cen', '1_v2', {'id': 'le_3', 'weight': 0.0}),
 ('1_cen', '1_v3', {'id': 'le_4', 'weight': 0.0}),
 ('2_cen', '2_v6', {'id': 'le_5', 'weight': 0.0}),
 ('2_cen', '2_v7', {'id': 'le_6', 'weight': 0.0}),
 ('2_cen', '2_v8', {'id': 'le_7', 'weight': 0.0}),
 ('2_cen', '2_v9', {'id': 'le_8', 'weight': 0.0}),
 ('1_v0', '1_v0_cp', {'id': 'ce_1', 'weight': 0.0}),
 ('1_v3', '1_v3_cp', {'id': 'ce_2', 'weight': 0.0}),
 ('1_v1', '1_v1_cp', {'id': 'ce_3', 'weight': 0.0}),
 ('2_v6', '2_v6_cp', {'id': 'ce_5', 'weight': 0.0}),
 ('2_v7', '2_v7_cp', {'id': 'ce_6', 'weight': 0.0}),
 ('2_v8', '2_v8_cp', {'id': 'ce_7', 'weight': 0.0}),
 ('1_v0_cp', 'n_26', {'id': 'cwe_1', 'weight': 1.3102878906544215}),
 ('1_v3_cp', 'n_26', {'id': 'cwe_2', 'weight': 1.0099553365085958}),
 ('1_v1_cp', 'n_20', {'id': 'cwe_3', 'weight': 0.7485869011242209}),
 ('2_v6_cp', 'n_25', {'id': 'cwe_5', 'weight': 1.2371480639423496})

In [12]:
G_loc = nx.DiGraph(edgelist_loc)
G_di = G.to_directed()
print(len(G.edges))
print(len(G_di.edges))
print(len(G_loc.edges))

G_locations = nx.compose(G_di, G_loc)
print(len(G_locations.edges))

47
94
26
120


In [13]:
gdf_edges_loc = pd.concat([gdf_edges, gdf_loc_edges_merged])

In [14]:
gdf_loc_edges_merged['geom_nd_st'] = gdf_loc_edges_merged.apply(lambda row: Point(row.geometry.coords[0]), axis=1)
gdf_loc_edges_merged['geom_nd_en'] = gdf_loc_edges_merged.apply(lambda row: Point(row.geometry.coords[1]), axis=1)

gdf_nodes_loc = pd.concat([gdf_loc_edges_merged[['nd_st', 'geom_nd_st']].rename(columns={'nd_st':'node_id', 'geom_nd_st':'geometry'}), 
           gdf_loc_edges_merged[['nd_en', 'geom_nd_en']].rename(columns={'nd_en':'node_id', 'geom_nd_en':'geometry'})])
gdf_nodes_loc.drop_duplicates(subset='node_id', inplace=True)
gdf_nodes_loc.reset_index(inplace=True, drop=True)

gdf_nodes_loc = pd.concat([gdf_nodes, gdf_nodes_loc])

c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / Pseudo-Mercator (the single non-null crs provided).
  warnings.warn(


# ISOCHRONES

In [15]:
from netana.tools.calc_tools import split_line_by_point

def calculate_travel_time_to_all_nodes(graph, f_weight, loc_node_list):
    result = nx.multi_source_dijkstra_path_length(graph, 
                                        sources = loc_node_list, 
                                        cutoff=None, 
                                        weight=f_weight)
    df = pd.DataFrame(result.items(), columns=['node_id', 'weight_to'])
    return df

def asign_weight_values_to_nodes_and_edges(gdf_nodes, gdf_edges, df_weight):
    df_nodes= gdf_nodes.merge(df_weight[['node_id', 'weight_to']], on='node_id', how='left')
    df_nodes= df_nodes[['node_id', 'geometry', 'weight_to']]

    df_edges = gdf_edges.merge(df_weight[['node_id', 'weight_to']], left_on='nd_st', right_on='node_id', how='left')
    df_edges = df_edges.merge(df_weight[['node_id', 'weight_to']], left_on='nd_en', right_on='node_id', how='left')
    df_edges.rename(columns={'weight_to_x':'weight_start', 'weight_to_y':'weight_end'}, inplace=True)
    df_edges.drop(columns=['node_id_x', 'node_id_y'], inplace=True)

    return df_nodes, df_edges

def create_service_area_by_weight_limit(gdf_edges, weight_limit):

    gdf_edges['cat_limit'] = 'out'
    gdf_edges.loc[(gdf_edges.weight_start < weight_limit) & (gdf_edges.weight_end < weight_limit), 'cat_limit'] = 'ok'
    df_edges_ok = gdf_edges.loc[gdf_edges.cat_limit == 'ok'].reset_index(drop=True)
   
    df_edges__from_start = gdf_edges[(gdf_edges.weight_start < weight_limit) & (gdf_edges.weight_end > weight_limit)].reset_index(drop=True)
    if len(df_edges__from_start) > 0:
        df_edges__from_start['cat_limit'] = 'start'
        df_edges__from_start['weight_delta'] = (weight_limit - df_edges__from_start['weight_start']) / (df_edges__from_start['weight_end'] - df_edges__from_start['weight_start'])
        df_edges__from_start['geometry'] = df_edges__from_start.apply(lambda row: split_line_by_point(row.geometry, row.weight_delta, 'start'), axis=1)

    df_edges_from_end = gdf_edges[(gdf_edges.weight_start > weight_limit) & (gdf_edges.weight_end < weight_limit)].reset_index(drop=True)
    if len(df_edges_from_end) > 0:
        df_edges_from_end['cat_limit'] = 'end'
        df_edges_from_end['weight_delta'] = (weight_limit - df_edges_from_end['weight_end']) / (df_edges_from_end['weight_start'] - df_edges_from_end['weight_end'])
        df_edges_from_end['geometry'] = df_edges_from_end.apply(lambda row: split_line_by_point(row.geometry, row.weight_delta, 'end'), axis=1)

    df_edges_weight_limit = pd.concat([df_edges_ok, df_edges__from_start, df_edges_from_end])

    df_edges_weight_limit.drop(columns=['cat_limit'], inplace=True)
    df_edges_weight_limit['weight_limit'] = weight_limit
    return df_edges_weight_limit

def calculate_service_edges(graph, gdf_nodes, gdf_edges, f_weight, v_weight_limit, gdf_locations):
    list_source_nodes = list(gdf_locations['node_id'])
    df_weight_times = calculate_travel_time_to_all_nodes(graph, f_weight, list_source_nodes)
    gdf_nodes, gdf_edges = asign_weight_values_to_nodes_and_edges(gdf_nodes, gdf_edges, df_weight_times)
    df_edges_weight_limit = create_service_area_by_weight_limit(gdf_edges, v_weight_limit)
    return df_edges_weight_limit

In [18]:
gdf_locations_edited = gdf_locations.copy()
gdf_locations_edited.loc[0, 'node_id'] = '1_cen'
gdf_locations_edited.loc[1, 'node_id'] = 'n_27'
gdf_locations_edited

,id,node_id,geometry
0,1,1_cen,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,2,n_27,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


In [19]:
df_edges_cost_limit_5_min = calculate_service_edges(G_locations, gdf_nodes_loc, gdf_edges_loc, 'weight', 3, gdf_locations_edited)
df_edges_cost_limit_5_min.to_file("./output/pruebas_2025/edges_iso.shp")
df_edges_cost_limit_5_min.head()

C:\Users\Compumar\AppData\Local\Temp\ipykernel_4244\1551923938.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_edges_cost_limit_5_min.to_file("./output/pruebas_2025/edges_iso.shp")


,id,nd_st,nd_en,weight,geometry,weight_start,weight_end,weight_delta,weight_limit
0,e_7,n_20,n_12,1.972933,"LINESTRING (-6546884.300 -3184165.694, -654685...",0.122379,1.224346,NaN,3
1,e_8,n_26,n_20,1.432667,"LINESTRING (-6546778.279 -3184183.149, -654688...",1.009955,0.122379,NaN,3
2,e_9,n_12,n_21,1.709733,"LINESTRING (-6546859.721 -3184019.784, -654683...",1.224346,2.934080,NaN,3
3,e_10,n_15,n_12,1.350000,"LINESTRING (-6546759.845 -3184036.398, -654685...",2.574346,1.224346,NaN,3
4,e_16,n_10,n_25,1.162133,"LINESTRING (-6546796.591 -3183610.196, -654671...",2.786667,1.624533,NaN,3


In [228]:
from netana.service_analysis.service_area_analysis import create_service_area
gdf_buffer = create_service_area(df_edges_cost_limit_5_min, 5, 100)
gdf_buffer.to_file("./notebook/zz_buffer.shp")
gdf_buffer

C:\Users\Compumar\AppData\Local\Temp\ipykernel_16300\2776910315.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_buffer.to_file("./notebook/zz_buffer.shp")


,geometry,id,nd_st,nd_en,weight,weight_start,weight_end,weight_limit
0,"MULTIPOLYGON (((-6546721.650 -3183661.908, -65...",e_19,n_21,n_17,11.859794,119.307678,107.447884,150


,geometry,id,nd_st,nd_en,weight,weight_start,weight_end,weight_limit
0,"MULTIPOLYGON (((-6546721.650 -3183661.908, -65...",e_19,n_21,n_17,11.859794,119.307678,107.447884,150
